In [1]:
import numpy as np
import gym

from torch.utils.tensorboard import SummaryWriter
import datetime
from collections import namedtuple
from collections import deque
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.utils.clip_grad import clip_grad_norm_


In [9]:
# Parameters
Box2DEnvName = 'BipedalWalker-v3'
SIZEofBATCH = 64
EPOCHS_PPO = 5
MAXIMUM_ITERATION = 25000
DEVICE = 'cpu'
GRADIENTCLIP = 0.1
SIZE_TRAJECTORY = 2048
GAE_LAMBDA = 0.90
EPSCLIP = 0.1
GAMMA = 0.98

In [11]:
# Testing the Hyperparameters
TEST_VIDEO_SAVE = True
N_ITER_TEST = 100
POLICY_LR = 0.0003
VALUE_LR = 0.001
EPISODESTESTED = 5
BESTRESULTSTESTED = -1e5
now = datetime.datetime.now()
TIMEDATE = "{}_{}.{}.{}".format(now.day, now.hour, now.minute, now.second)
MODELLOADED = False
NameCheckpoint = "checkpoints/..."

In [4]:
class A2CPOLICY(nn.Module):
   
    def __init__(self, inputShape, NActions):
        super(A2CPOLICY, self).__init__()

        self.lp = nn.Sequential(
            nn.Linear(inputShape[0], 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU())

        self.mean_l = nn.Linear(32, NActions[0])
        self.mean_l.weight.data.mul_(0.1)

        self.var_l = nn.Linear(32, NActions[0])
        self.var_l.weight.data.mul_(0.1)

        self.logstd = nn.Parameter(torch.zeros(NActions[0]))

    def forward(self, x):
        ot_n = self.lp(x.float())
        return F.tanh(self.mean_l(ot_n))

In [5]:
class A2Cvalue(nn.Module):
    
    def __init__(self, inputShape):
        super(A2Cvalue, self).__init__()

        self.lp = nn.Sequential(
            nn.Linear(inputShape[0], 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 1))


    def forward(self, x):
        return self.lp(x.float())

In [6]:
class Environment:
  
    gameReward = 0
    lastGameReward = 0
    gameN = 0
    lastGamesRews = [-200]
    n_iter = 0

    def __init__(self, Box2DEnvName, Nsteps, GAMMA, GAE_LAMBDA, save_video=False):
        super(Environment, self).__init__()

        # create the new environment
        self.Environment = gym.make(Box2DEnvName)
        self.observation = self.Environment.reset()

        self.Nsteps = Nsteps
        self.Naction = self.Environment.action_space.shape
        self.Nobservation = self.env.observation_space.shape[0]
        self.GAMMA = GAMMA
        self.GAE_LAMBDA = GAE_LAMBDA

In [7]:

def steps(self, agentPolicy, agentValue):
     
        memories = []
        for s in range(self.Nsteps):
            self.n_iter += 1

           
            agentMean = agent_policy(torch.tensor(self.observation))

           
            logstd = agent_policy.logstd.data.cpu().numpy()
            Action = ag_mean.data.cpu().numpy() + np.exp(logstd) * np.random.normal(size=logstd.shape)
            Action = np.clip(action, -1, 1)

            stateValue = float(agentValue(torch.tensor(self.observation)))

       
            new_observation, reward, done, _ = self.Environment.step(Action)

        
            if done:
            
                memories.append(Memory(observation=self.observation, action=action, new_observation=new_observation, reward=0, done=done, value=state_value, adv=0))
            else:
                memories.append(Memory(obs=self.observation, Action=Action, new_observation=new_observation, reward=reward, done=done, value=stateValue, adv=0))


            self.game_rew += reward
            self.obs = new_observation

            if done:
                print('#####',self.game_n, 'rew:', int(self.game_rew), int(np.mean(self.last_games_rews[-100:])), np.round(reward,2), self.n_iter)

               
                self.obs = self.env.reset()
                self.last_game_rew = self.game_rew
                self.game_rew = 0
                self.game_n += 1
                self.n_iter = 0
                self.last_games_rews.append(self.last_game_rew)

 
        return self.generalized_advantage_estimation(memories)

def generalized_advantage_estimation(self, memories):
        
        upd_memories = []
        run_add = 0

        for t in reversed(range(len(memories)-1)):
            if memories[t].done:
                run_add = memories[t].reward
            else:
                sigma = memories[t].reward + self.gamma * memories[t+1].value - memories[t].value
                run_add = sigma + run_add * self.gamma * self.gae_lambda

           
          
            upd_memories.append(Memory(obs=memories[t].obs, action=memories[t].action, new_obs=memories[t].new_obs, reward=run_add + memories[t].value, done=memories[t].done, value=memories[t].value, adv=run_add))

        return upd_memories[::-1]


def log_policy_prob(mean, std, actions):
    
    act_log_softmax = -((mean-actions)**2)/(2*torch.exp(std).clamp(min=1e-4)) - torch.log(torch.sqrt(2*math.pi*torch.exp(std)))
    return act_log_softmax

def compute_log_policy_prob(memories, nn_policy, device):
    
    n_mean = nn_policy(torch.tensor(np.array([m.obs for m in memories], dtype=np.float32)).to(device))
    n_mean = n_mean.type(torch.DoubleTensor)
    logstd = agent_policy.logstd.type(torch.DoubleTensor)

    actions = torch.DoubleTensor(np.array([m.action for m in memories])).to(device)

    return log_policy_prob(n_mean, logstd, actions)

def clipped_PPO_loss(memories, nn_policy, nn_value, old_log_policy, adv, epsilon, writer, device):

   
    rewards = torch.tensor(np.array([m.reward for m in memories], dtype=np.float32)).to(device)
    value = nn_value(torch.tensor(np.array([m.obs for m in memories], dtype=np.float32)).to(device))
    
    # Value loss
    vl_loss = F.mse_loss(value.squeeze(-1), rewards)

    new_log_policy = compute_log_policy_prob(memories, nn_policy, device)
    rt_theta = torch.exp(new_log_policy - old_log_policy.detach())

    adv = adv.unsqueeze(-1) # add a dimension because rt_theta has shape: [batch_size, n_actions]
    pg_loss = -torch.mean(torch.min(rt_theta*adv, torch.clamp(rt_theta, 1-epsilon, 1+epsilon)*adv))

    return pg_loss, vl_loss

def test_game(tst_env, agent_policy, test_episodes):
   
    reward_games = []
    steps_games = []
    for _ in range(test_episodes):
        obs = tst_env.reset()
        rewards = 0
        steps = 0
        while True:
            ag_mean = agent_policy(torch.tensor(obs))
            action = np.clip(ag_mean.data.cpu().numpy().squeeze(), -1, 1)

            next_obs, reward, done, _ = tst_env.step(action)
            steps += 1
            obs = next_obs
            rewards += reward

            if done:
                reward_games.append(rewards)
                steps_games.append(steps)
                obs = tst_env.reset()
                break

    return np.mean(reward_games), np.mean(steps_games)


Memory = namedtuple('Memory', ['obs', 'action', 'new_obs', 'reward', 'done', 'value', 'adv'],rename=False)

In [8]:
!pip install box2d-py
!pip install gym[all]
!pip install gym[Box_2D]
!pip install git+https://github.com/AI4Finance-LLC/ElegantRL.git

import gym

if __name__ == '__main__':
  
    env = Environment(Box2DEnvName, SIZE_TRAJECTORY, GAMMA, GAE_LAMBDA)

    writer_name = 'PPO_'+ENV_NAME+'_'+date_time+'_'+str(POLICY_LR)+'_'+str(VALUE_LR)+'_'+str(TRAJECTORY_SIZE)+'_'+str(BATCH_SIZE)
    writer = SummaryWriter(log_dir='content/runs/'+writer_name)

   
    test_env = gym.make(Box2DEnvName)
    if save_video_test:
        test_env = gym.wrappers.Monitor(test_env,  "VIDEOS/TEST_VIDEOS_"+writer_name, video_callable=lambda episode_id: episode_id%10==0)


    agent_policy = A2C_policy(test_env.observation_space.shape, test_env.action_space.shape).to(device)
    agent_value = A2C_value(test_env.observation_space.shape).to(device)


    optimizer_policy = optim.Adam(agent_policy.parameters(), lr=POLICY_LR)
    optimizer_value = optim.Adam(agent_value.parameters(), lr=VALUE_LR)

    if load_model:
        print('> Loading checkpoint {}'.format(checkpoint_name))
        checkpoint = torch.load(checkpoint_name)
        agent_policy.load_state_dict(checkpoint['agent_policy'])
        agent_value.load_state_dict(checkpoint['agent_value'])
        optimizer_policy.load_state_dict(checkpoint['optimizer_policy'])
        optimizer_value.load_state_dict(checkpoint['optimizer_value'])


    experience = []
    n_iter = 0

    while n_iter < MAX_ITER:
        n_iter += 1

        batch = env.steps(agent_policy, agent_value)

      
        old_log_policy = compute_log_policy_prob(batch, agent_policy, device)


        batch_adv = np.array([m.adv for m in batch])
        
        batch_adv = (batch_adv - np.mean(batch_adv)) / (np.std(batch_adv) + 1e-7)
        batch_adv = torch.tensor(batch_adv).to(device)

       
        pol_loss_acc = []
        val_loss_acc = []

       
        for s in range(PPO_EPOCHS):
         
            for mb in range(0, len(batch), BATCH_SIZE):
                mini_batch = batch[mb:mb+BATCH_SIZE]
                minib_old_log_policy = old_log_policy[mb:mb+BATCH_SIZE]
                minib_adv = batch_adv[mb:mb+BATCH_SIZE]

              
                pol_loss, val_loss = clipped_PPO_loss(mini_batch, agent_policy, agent_value, minib_old_log_policy, minib_adv, CLIP_EPS, writer, device)

               
                optimizer_policy.zero_grad()
                pol_loss.backward()
                optimizer_policy.step()

                
                optimizer_value.zero_grad()
                val_loss.backward()
                optimizer_value.step()

                pol_loss_acc.append(float(pol_loss))
                val_loss_acc.append(float(val_loss))

       
        writer.add_scalar('pg_loss', np.mean(pol_loss_acc), n_iter)
        writer.add_scalar('vl_loss', np.mean(val_loss_acc), n_iter)
        writer.add_scalar('rew', env.last_game_rew, n_iter)
        writer.add_scalar('10rew', np.mean(env.last_games_rews[-100:]), n_iter)

        
        if n_iter % N_ITER_TEST == 0:
            test_rews, test_stps = test_game(test_env, agent_policy, test_episodes)
            print(' > Testing..', n_iter,test_rews, test_stps)
         
            if test_rews > best_test_result:
                torch.save({
                    'agent_policy': agent_policy.state_dict(),
                    'agent_value': agent_value.state_dict(),
                    'optimizer_policy': optimizer_policy.state_dict(),
                    'optimizer_value': optimizer_value.state_dict(),
                    'test_reward': test_rews
                }, 'checkpoints/checkpoint_'+writer_name+'.pth.tar')
                best_test_result = test_rews
                print('=> Best test!! Reward:{:.2f}  Steps:{}'.format(test_rews, test_stps))

            writer.add_scalar('test_rew', test_rews, n_iter)


    writer.close()

     |████████████████████████████████| 448 kB 5.6 MB/s 
     |████████████████████████████████| 120 kB 5.2 MB/s 
     |████████████████████████████████| 206 kB 33.9 MB/s 
  ERROR: Failed building wheel for mujoco-py
  Running setup.py clean for mujoco-py
Failed to build mujoco-py
    Running setup.py install for mujoco-py ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-lzqhj3fy/mujoco-py_d05817ba70274bbda7b9555cde35c40a/setup.py'"'"'; __file__='"'"'/tmp/pip-install-lzqhj3fy/mujoco-py_d05817ba70274bbda7b9555cde35c40a/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-q8h4irov/install-record.txt --single-version-externally-mana

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


AttributeError: ignored